# B2B Agent Workflow

This notebook demonstrates the B2B Agent multi-agent workflow system for lead finding, collection, and enrichment.



# Libs

In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional
import random
from langchain_core.messages import ToolMessage

# Code

In [ ]:
class Lead(BaseModel):
    """Structured lead with enrichment fields"""
    company: str
    industry: str
    employee_count: int
    revenue_musd: float
    
    # Enrichment fields - initially None
    website: Optional[str] = None
    last_year_profit: Optional[float] = None      
    last_quarter_ebitda: Optional[float] = None
    stock_variation_3m: Optional[float] = None
    
    def needs_enrichment(self) -> bool:
        """Check if lead still needs enrichment"""
        return (
            self.website is None or
            self.last_year_profit is None or
            self.last_quarter_ebitda is None or
            self.stock_variation_3m is None
        )

class lead_completed(BaseModel):
    company: str = Field(..., description="Name of the company")
    industry: str = Field(..., description="Industry sector of the company")
    employee_count: int = Field(..., description="Number of employees at the company")
    revenue_musd: float = Field(..., description="Annual revenue in millions of USD")
    website: str = Field(..., description="Official website URL of the company")
    last_year_profit: float = Field(..., description="Company's profit for the last fiscal year in millions of USD")
    last_quarter_ebitda: float = Field(..., description="Company's EBITDA for the last quarter in millions of USD")
    stock_variation_3m: float = Field(..., description="Stock price variation over the last 3 months in percentage")
    

# Base class to define a state
class State(BaseModel):
    messages: Annotated[list, add_messages]
    leads: list[Lead] = [] 
    filtered_leads: list[Lead] = [] 
    # enriched_lead: Optional[Lead] = None
    next_action: str = ""

In [ ]:
def get_leads(state: State) -> dict:
    # Simulating the leads
    leads = [
        Lead(
            company="Americanas S.A.",
            industry="Marketplace",
            employee_count=1200,
            revenue_musd=12.4
        ),
        Lead(
            company="Grupo Madero",
            industry="food",
            employee_count=400,
            revenue_musd=0.1
        ),
        Lead(
            company="Grupo Boticário",
            industry="Beauty & Personal Care",
            employee_count=250,
            revenue_musd=1
        )
    ]
    
    # Return updates as a dict (LangGraph will merge with state)
    return {
        "leads": [lead.model_dump() for lead in leads],
        "messages": [{"role": "assistant", "content": f"Found {len(leads)} leads"}]
    }

In [ ]:
def triage(state: State) -> dict:
    print("TRIAGE")
    # Access leads from state
    leads = state.leads if hasattr(state, 'leads') else []

    # Apply simple rules to filter leads
    filtered = []
    for lead in leads:
        if (
            lead.employee_count >= 0 and
            lead.revenue_musd >= 0 and
            lead.industry != "logistics"
        ):
            filtered.append(lead)
    
    print("""=== FILTERED LEADS ===""")
    for lead in filtered:
        print(lead)
    # Return updates as a dict
    return {
        "filtered_leads": [lead.model_dump() for lead in filtered],
        "messages": [{"role": "assistant", "content": f"Filtered to {len(filtered)} qualified leads: {filtered}"}]
    }

## Graph

In [ ]:
# Starting a new graph
graph_builder = StateGraph(State)

In [ ]:
# Create nodes
graph_builder.add_node("lead_finder", get_leads);
graph_builder.add_node("triage", triage);

In [ ]:
# Edges
graph_builder.add_edge(START, "lead_finder");
graph_builder.add_edge("lead_finder", "triage");
graph_builder.add_edge("triage", END);

In [ ]:
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke(State(messages=[{"role": "user", "content": "Hello"}]))

In [ ]:
# def chat(user_input: str, history):
#     message = {"role": "user", "content": user_input}
#     messages = [message]
#     state = State(messages=messages)
#     result = graph.invoke(state)
#     print(result)
#     return result["messages"][-1].content


# gr.ChatInterface(chat, type="messages").launch()

## Graph + LLM

In [ ]:
# Loading env variables
load_dotenv(override=True)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
def chatbot_node(old_state: State) -> dict:
    # Ask LLM to analyze intent
    system_msg = {
        "role": "system",
        "content": "If user wants to find leads, respond with EXACTLY 'FIND_LEADS'. Otherwise chat normally."
    }
    messages = [system_msg] + old_state.messages
    response = llm.invoke(messages)
    
    # Check if LLM wants to trigger lead finding
    if "FIND_LEADS" in response.content:
        # Don't show "FIND_LEADS" to user - show a friendly transitional message
        return {
            "messages": [{"role": "assistant", "content": "Great! Let me find some qualified leads for you..."}],
            "next_action": "find_leads"
        }
    else:
        # Normal chat - show the actual LLM response
        return {
            "messages": [{"role": "assistant", "content": response.content}],
            "next_action": "end"
        }

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool
from langgraph.prebuilt import ToolNode, tools_condition


# Tool to find more information for a lead
serper_search = GoogleSerperAPIWrapper()

search_tool = Tool(
    name="search_company_info",
    description="Search the web for detailed information about a company including recent news, technologies used, partnerships, and business updates. Use this when you need more context about a lead company.",
    func=serper_search.run
)

In [ ]:
# search_tool.invoke("Tell me the Brazilian company Americanas S.A. revenue for 2024")

In [ ]:
tools = [search_tool]

In [ ]:
def enrich_leads(state: State) -> dict:
    """LLM decides if leads need enrichment using the search tool."""
    filtered = state.filtered_leads
    
    if not filtered:
        return {"filtered_leads": []}
    
    # Check if any leads still need enrichment
    leads_needing_enrichment = [lead for lead in filtered if lead.needs_enrichment()]
    print("-"*100)
    print(leads_needing_enrichment)
    print("-"*100)
    
    if not leads_needing_enrichment:
        # All leads are enriched, end the loop
        return {
            "messages": [{"role": "assistant", "content": "All leads have been enriched!"}]
        }
    
    # Only enrich one company at a time to avoid repetition
    lead_to_enrich = leads_needing_enrichment[0]

    # ⭐ KEY FIX: Check if we just received tool results
    last_message = state.messages[-1] if state.messages else None
    
    if isinstance(last_message, ToolMessage):
        # We have tool results - don't request tools again, let it go to update_lead
        print(f"✓ Tool results received for {lead_to_enrich.company}, proceeding to update...")
        return {
            "messages": [{"role": "assistant", "content": f"Processing results for {lead_to_enrich.company}"}],
            # "enriched_lead": lead_to_enrich
        }
    
    system_prompt = f"""You are enriching lead data for {lead_to_enrich.company}.
    
    Current data: {lead_to_enrich.model_dump_json()}
    
    Use search_company_info to find ONLY the missing fields. Be specific in your search query.
    After getting results, extract the relevant information clearly."""
    
    messages = state.messages + [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Find missing information for {lead_to_enrich.company}"}
    ]
    
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(messages)
    
    return {
        "messages": [response],
        # "enriched_lead": lead_to_enrich
    }

In [ ]:
def update_lead(state: State) -> dict:
    print("UPDATE LEAD")
    # Find the first lead that needs enrichment (this is the one we just got results for)
    lead_to_update = next((l for l in state.filtered_leads if l.needs_enrichment()), None)
    
    if not lead_to_update:
        return {}

    extractor = llm.with_structured_output(lead_completed)

    prompt = f"""
    Combine the existing lead and the search results, and output a full LeadCompleted object.

    Existing lead: {lead_to_update.model_dump_json()}
    Search results: {state.messages[-1].content}
    """

    enriched = extractor.invoke([{"role": "user", "content": prompt}])
    filtered = state.filtered_leads

    updated = [
        enriched.model_dump() if lead.company == enriched.company else lead.model_dump()
        for lead in filtered
    ]

    return {"filtered_leads": updated}

In [ ]:
def should_continue(state: State):
    return (
        "enricher"
        if any(l.needs_enrichment() for l in state.filtered_leads)
        else END
    )

In [ ]:
def generate_summary(state: State) -> dict:
    """Generate natural language summary of results"""
    filtered = state.filtered_leads if hasattr(state, 'filtered_leads') else []
    
    system_msg = {
        "role": "system",
        "content": f"""Summarize these B2B leads in a friendly way:
        Initial informations:
        {filtered}

        # Instructions
        - Use markdown to format the summary
        """
    }
    
    messages = [system_msg] + state.messages
    response = llm.invoke(messages)
    
    return {
        "messages": [{"role": "assistant", "content": response.content}]
    }

In [ ]:
# Starting a new graph
graph_builder = StateGraph(State)

In [ ]:
# Create nodes
graph_builder.add_node("lead_finder", get_leads);
graph_builder.add_node("triage", triage);
graph_builder.add_node("chatbot", chatbot_node);
graph_builder.add_node("summary", generate_summary);

In [ ]:
graph_builder.add_node("enricher", enrich_leads)
graph_builder.add_node("tools", ToolNode(tools=tools))
graph_builder.add_node("update_lead", update_lead)

In [ ]:
# Routing logic
def route_after_chatbot(state: State) -> str:
    # Access the attribute directly (Pydantic BaseModel)
    next_action = state.next_action
    print(f"Routing decision: {next_action}")
    return "lead_finder" if next_action == "find_leads" else END

In [ ]:
# Edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", route_after_chatbot, {
    "lead_finder": "lead_finder",
    END: END
})
graph_builder.add_edge("lead_finder", "triage")
graph_builder.add_edge("triage", "enricher")

graph_builder.add_conditional_edges(
    "enricher", 
    tools_condition,
    {
        "tools": "tools",        # If tool calls exist, execute them
        "__end__": "update_lead" # Otherwise, go to update_lead
    }
)

# After tools execute, back to enricher for next iteration
graph_builder.add_edge("tools", "enricher")

# After update, check if more enrichment needed
graph_builder.add_conditional_edges(
    "update_lead",
    should_continue,
    {
        "enricher": "enricher",  # Continue with next lead
        END: "summary"           # ← Changed: Go to summary instead of END
    }
)


In [ ]:
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input}
    messages = [message]
    state = State(messages=messages)
    result = graph.invoke(state)
    print("State %",state)
    print("-"*100)
    print("Result %",result)
    return result["messages"][-1].content


gr.ChatInterface(
    chat, 
    type="messages",
    title="B2B Lead Generation Assistant",
    description="Ask me to find and qualify B2B leads!"
).launch()